# 13.基础Actor-Critic算法
> **Actor-Critic（执行者-评论者）** 算法结合了 **基于值** 的学习方法（如Q学习）和 **基于策略** 的学习方法（如策略梯度）。这种方法将智能体的学习过程分为两个部分： **Actor（执行者）** 和 **Critic（评论者）**，每个部分负责不同的任务：**Actor** 通过策略采取行动，**Critic** 通过价值函数给 **Actor** 反馈当前状态或状态-动作对的“好坏”。其本质上是 **基于策略** 的算法，算法的目标是优化一个带参数的策略，只是会额外学习价值函数，从而帮助策略函数更好地学习。
> **Actor-Critic** 是囊括一系列算法的**整体架构**，目前很多高效的前沿算法都属于 **Actor-Critic** 算法。
> **Actor-Critic** 算法最早的理论基础和框架由 Bertsekas 和 Tsitsiklis 在他们的经典书籍《Neuro-Dynamic Programming》（1989年）中提出，正式提出在[Actor-Critic Algorithms（2000年）](https://proceedings.neurips.cc/paper_files/paper/1999/file/6449f44a102fde848669bdd9eb6b76fa-Paper.pdf)论文中。

> 本节会学习一种最基础的 **Actor-Critic** 算法：

## 13.1 Actor-Critic原理
> 前面学习到策略梯度，可以表示为：
$$grad=\mathbb{E}\left[\sum_{t=0}^T\psi_t\nabla_\theta\log\pi_\theta(a_t|s_t)\right]$$
> 其中，$\psi_t$有很多种形式：
$$\begin{aligned}
 & 1.\sum_{t^{\prime}=0}^{T}\gamma^{t^{\prime}}r_{t^{\prime}}:\text{轨迹的总回报;} \\
 & 2.\sum_{t^{\prime}=t}^T\gamma^{t^{\prime}-t}r_{t^{\prime}}:\text{动作}a_t\text{之后的回报;} \\
 & 3.\sum_{t^{\prime}=t}^T\gamma^{t^{\prime}-t}r_{t^{\prime}}-b(s_t):\text{基准线版本的改进;} \\
 & 4.Q^{\pi_{\theta}}(s_{t},a_{t}):\text{动作价值函数;} \\
 & 5.A^{\pi_{\theta}}(s_{t},a_{t}):\text{优势函数;} \\
 & 6.r_t+\gamma V^{\pi_\theta}(s_{t+1})-V^{\pi_\theta}(s_t):\text{时序差分残差.}
\end{aligned}$$

### 本节采用**形式6**，$\delta_t=\underbrace{r_t+\gamma V(s_{t+1})}_{\text{bootstrapped 回报}}-\underbrace{V(s_t)}_{\text{当前估计}}$，它衡量着**当前估计**和**实际回报**之间的差异
> Bootstrap：用当前估计的未来回报替代未来真实回报

> 这里有一个非常需要**注意**的点：前5种方式都是对应价值函数越大越好，但第6种形式看起来“误差越小越好”，这改变了最大化期望回报的优化方向了吗？实际上仍然是最大化期望回报： 因为从**Critic（评论者）** 角度看，**Actor（执行者）** 执行了某个动作，如果该动作与之前比确实好价值高，此时正误差就应该是大的，于是**Actor（策略网络）** 向梯度方向增加好动作的概率，同时**Critic（价值网络）** 也会最小化该误差，定义**损失函数**为：
$$\mathcal{L}(\omega)=\frac{1}{2}(r+\gamma V_\omega(s_{t+1})-V_\omega(s_t))^2$$
> **Critic（价值网络）** 梯度：
$$\nabla_\omega\mathcal{L}(\omega)=-(r+\gamma V_\omega(s_{t+1})-V_\omega(s_t))\nabla_\omega V_\omega(s_t)=-\delta_t$$
> 且更新**价值参数**：$w=w+\alpha_\omega\sum_t\delta_t\nabla_\omega V_\omega(s_t)$，使得下一次执行该动作后误差变小，如此在**Critic（评论者）** 看来价值也相对没那么高，稳定了动作概率的提升。

> 从这也可以看出 **时序差分残差** 可以与 **优势函数A** 相互替换，两者都衡量了不同动作的好坏。


> 也可以从另一个角度看:
$$\nabla J(\theta)\approx\mathbb{E}_{s_t,a_t}\left[\nabla\log\pi_\theta(a_t|s_t)\cdot(r_t+\gamma V_\phi(s_{t+1})-V_\phi(s_t))\right]$$
> 减去$V_\phi(s_t)$ 仅相当于减去一个基线项，不改变期望，只是降低方差

## 13.2 Actor-Critic 代码实践
> 仍在 **车杆环境** 下进行 Actor-Critic 算法的实验

导入相关库：

In [1]:
# 基本库
import numpy as np
from tqdm import tqdm
from utils.smoothing import moving_average
# 神经网络
import torch
import torch.nn.functional as F
# Gymnasium 是一个用于开发和测试强化学习算法的工具库，为 OpenAI Gym 的更新版本（2021迁移开发）
import gymnasium as gym

定义策略网络PolicyNet（与 REINFORCE 算法一样）：

In [2]:
class PolicyNet(torch.nn.Module):
    def __init__(self, state_dim, hidden_dim, action_dim):
        super(PolicyNet, self).__init__()
        self.fc1 = torch.nn.Linear(state_dim, hidden_dim)
        self.fc2 = torch.nn.Linear(hidden_dim, action_dim)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        return F.softmax(self.fc2(x), dim=1)

定义价值网络ValueNet：

In [3]:
class ValueNet(torch.nn.Module):
    def __init__(self, state_dim, hidden_dim):
        super(ValueNet, self).__init__()
        self.fc1 = torch.nn.Linear(state_dim, hidden_dim)
        self.fc2 = torch.nn.Linear(hidden_dim, 1)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        return self.fc2(x)

定义 Actor-Critic 算法：

In [4]:
class ActorCritic:
    def __init__(self, state_dim, hidden_dim, action_dim, actor_lr, critic_lr, gamma, device):
        
        self.actor = PolicyNet(state_dim, hidden_dim, action_dim).to(device)  # 策略网络
        self.critic = ValueNet(state_dim, hidden_dim).to(device)  # 价值网络
        self.actor_optimizer = torch.optim.Adam(self.actor.parameters(), lr=actor_lr)  # 策略网络优化器
        self.critic_optimizer = torch.optim.Adam(self.critic.parameters(), lr=critic_lr)  # 价值网络优化器
        self.gamma = gamma
        self.device = device

    def take_action(self, state):
        state = torch.tensor(np.array([state]), dtype=torch.float).to(self.device)
        probs = self.actor(state)
        action_dist = torch.distributions.Categorical(probs)
        action = action_dist.sample()
        return action.item()

    def update(self, transition_dict):
        states_np = np.array(transition_dict['states'])  # 转换成统一的大 np.ndarray，PyTorch更高效处理
        # 之前的算法代码中没有,是因为已在replay_buffer方法中转换过了
        states = torch.tensor(states_np, dtype=torch.float).to(self.device)
        actions = torch.tensor(transition_dict['actions']).view(-1, 1).to(self.device)
        rewards = torch.tensor(transition_dict['rewards'], dtype=torch.float).view(-1, 1).to(self.device)
        next_states_np = np.array(transition_dict['next_states'])  # 转换成统一的大 np.ndarray，PyTorch更高效处理
        next_states = torch.tensor(next_states_np, dtype=torch.float).to(self.device)
        dones = torch.tensor(transition_dict['dones'], dtype=torch.float).view(-1, 1).to(self.device)

        td_target = rewards + self.gamma * self.critic(next_states) * (1 - dones)  # 时序差分目标
        td_delta = td_target - self.critic(states)                                 # 时序差分误差
        log_probs = torch.log(self.actor(states).gather(1, actions))
        actor_loss = torch.mean(-log_probs * td_delta.detach())  # 会“向前”计算价值网络的梯度，需要 .detach() 截断 
        critic_loss = torch.mean(F.mse_loss(self.critic(states), td_target.detach()))  # 均方误差损失函数
        self.actor_optimizer.zero_grad()  
        self.critic_optimizer.zero_grad()
        actor_loss.backward()  # 计算策略网络的梯度
        critic_loss.backward()  # 计算价值网络的梯度
        self.actor_optimizer.step()  # 更新策略网络的参数
        self.critic_optimizer.step()  # 更新价值网络的参数

环境设置（'CartPole-v1'）：

In [5]:
np.random.seed(0)    # 设置 NumPy 的随机种子
torch.manual_seed(0) # 设置 PyTorch CPU 随机种子
torch.cuda.manual_seed_all(0) # 设置 PyTorch GPU 随机种子, 由于GPU并行性, 只能极大减小偏差

env = gym.make('CartPole-v1')  # CartPole-v1 最大回合步数修改到了500步(v0为200)
#env = env.unwrapped # 获取原始环境（绕过 TimeLimit 包装器）解除最大步数500限制
env.reset(seed=0)   # 环境通常依赖于其他随机数生成器来初始化状态、进行探索(推荐位于以上随机之后)
print("Environment spec:", env.spec)

Environment spec: EnvSpec(id='CartPole-v1', entry_point='gymnasium.envs.classic_control.cartpole:CartPoleEnv', reward_threshold=475.0, nondeterministic=False, max_episode_steps=500, order_enforce=True, disable_env_checker=False, kwargs={}, namespace=None, name='CartPole', version=1, additional_wrappers=(), vector_entry_point='gymnasium.envs.classic_control.cartpole:CartPoleVectorEnv')


超参数设置：

In [6]:
state_dim = env.observation_space.shape[0]
hidden_dim = 128
action_dim = env.action_space.n
actor_lr = 1e-3
critic_lr = 1e-2
gamma = 0.98
device = torch.device("cuda") if torch.cuda.is_available() else torch.device( "cpu")
agent = ActorCritic(state_dim, hidden_dim, action_dim, actor_lr, critic_lr, gamma, device)

num_episodes = 1000

测试与训练:
> **在线策略学习训练代码**之后将包装到utils：*from utils.training import train_on_policy_agent*

In [7]:
return_list = []
for i in range(10):
    with tqdm(total=int(num_episodes / 10), desc='Iteration %d' % i) as pbar:
        for i_episode in range(int(num_episodes / 10)):
            episode_return = 0
            transition_dict = {
                'states': [],
                'actions': [],
                'next_states': [],
                'rewards': [],
                'dones': []
            }
            state, info = env.reset()  # 测试阶段(调整参数与对比算法)种子应固定; 训练阶段不固定，提高泛化能力
            done = False
            truncated = False
            while not (done or truncated):  # 杆子倒下或达到最大步数
                action = agent.take_action(state)
                next_state, reward, done, truncated, _ = env.step(action)  # Gymnasium返回值不一样
                transition_dict['states'].append(state)
                transition_dict['actions'].append(action)
                transition_dict['next_states'].append(next_state)
                transition_dict['rewards'].append(reward)
                transition_dict['dones'].append(done)
                state = next_state
                episode_return += reward
            return_list.append(episode_return)
            agent.update(transition_dict)
            
            if (i_episode+1) % 10 == 0:
                pbar.set_postfix({'episode': '%d' % (num_episodes/10 * i + i_episode+1), 'return': '%.3f' % np.mean(return_list[-10:])})
            pbar.update(1)

Iteration 9: 100%|██████████| 100/100 [01:11<00:00,  1.40it/s, episode=1000, return=500.000]


绘图：

In [8]:
import pandas as pd
episodes_list = list(range(len(return_list)))
mv_return = moving_average(return_list, 9)
# 创建 DataFrame
df1 = pd.DataFrame({'Episodes': episodes_list, 'Returns': return_list})
df2 = pd.DataFrame({'Episodes': episodes_list, 'Returns': mv_return})
# 保存为 CSV 文件
df1.to_csv('AC_returns_data.csv', index=False)
df2.to_csv('AC_mv_returns_data.csv', index=False)

In [1]:
import plotly.graph_objects as go
import pandas as pd
df = pd.read_csv('./Data/AC_returns_data.csv')  # 从 CSV 文件中读取数据
fig = go.Figure()
fig.add_trace(go.Scatter(x=df['Episodes'], y=df['Returns'], mode='lines', name='Returns'))
fig.update_layout(
    title='AC on CartPole-v1',
    xaxis_title='Episodes',
    yaxis_title='Returns',
    showlegend=True
)
fig.show()

In [2]:
import plotly.graph_objects as go
import pandas as pd
df = pd.read_csv('./Data/AC_mv_returns_data.csv')  # 从 CSV 文件中读取数据
fig = go.Figure()
fig.add_trace(go.Scatter(x=df['Episodes'], y=df['Returns'], mode='lines', name='Returns'))
fig.update_layout(
    title='AC on CartPole-v1',
    xaxis_title='Episodes',
    yaxis_title='Returns',
    showlegend=True
)
fig.show()

> 可以发现，相比 **REINFORCE**，**Actor-Critic** 更快收敛到了最优策略，且训练过程稳定，抖动情况有明显的改进，这说明价值函数的引入减小了方差。
需要说明的是，之后将要学习的 **TRPO、PPO、DDPG、SAC** 等深度强化学习算法都是在 **Actor-Critic 框架** 下进行发展的。